# Actividad legislativa vigente
## Preprocesamiento

In [58]:
import pandas as pd
import re
import numpy as np
import warnings
warnings.filterwarnings('ignore')

def sacaLegislatura(string):
    if pd.isnull(string):
        return ''
    result = re.search('por la (.*) Legislatura', string)
    return result.group(1)

def sacaId(string):
    if pd.isnull(string):
        return ''
    result = re.search('Referencia=(.*)&quot', string)
    if not result:
        return None
    return result.group(1)

def split(string, split_str, index):
    if split_str not in string:
        return string
    if len(string.split(split_str)) < (index + 1):
        return string
    return string.split(split_str)[index]

perfiles = pd.read_csv('data/perfiles.csv')

perfiles = perfiles.dropna(how='all')
perfiles = perfiles.fillna('')

perfiles = perfiles[perfiles['links_Estatus'] == 'Activo'].reset_index()
    
perfiles['legislatura_actividad'] = perfiles['links_Actividad'].apply(sacaLegislatura)
perfiles['sil_id'] = perfiles['links_url'].apply(lambda x: x.split('&Referencia=')[1]).astype(int)

perfiles['nombre'] = perfiles['links_Nombre'].apply(lambda x: x.split(',')[1])
perfiles['apellido'] = perfiles['links_Nombre'].apply(lambda x: x.split(',')[0])

perfiles['suplente_nombre'] = perfiles['links_Suplente'].apply(lambda x: split(x, ',', 1))
perfiles['suplente_apellido'] = perfiles['links_Suplente'].apply(lambda x: split(x, ',', 0))

perfiles['fecha_nacimiento'] = perfiles['links_Nacimiento'].apply(lambda x: x.split('\n')[0].replace('Fecha:', ''))
perfiles['fecha_nacimiento'] = perfiles['fecha_nacimiento'].apply(lambda x: x if 'N/A' not in x else '')
perfiles['entidad_nacimiento'] = perfiles['links_Nacimiento'].apply(lambda x: x.split('\n')[1].replace('Entidad:', ''))
perfiles['ciudad_nacimiento'] = perfiles['links_Nacimiento'].apply(lambda x: x.split('\n')[2].replace('Ciudad:', ''))

perfiles['hasta_mandato'] = perfiles['links_Periodo_de_Legislatura'].apply(lambda x: x.split('al')[1].replace(' ', ''))
perfiles['desde_mandato'] = perfiles['links_Periodo_de_Legislatura'].apply(lambda x: x.split('al')[0].replace('Del', '').replace(' ', ''))

perfiles['entidad_actividad'] = perfiles['links_Zona'].apply(lambda x: split(x, '\n', 0).replace('Entidad:', ''))
perfiles['entidad_actividad'] = perfiles['entidad_actividad'].apply(lambda x: x if 'N/A' not in x else '')
perfiles['distrito_actividad'] = perfiles['links_Zona'].apply(lambda x: split(x, '\n', 1).replace('Distrito:', '').split(' (')[0] if 'Distrito' in x else '')
perfiles['municipalidad_actividad'] = perfiles['links_Zona'].apply(lambda x: split(split(x, '\n', 1).replace('Distrito:', ''), ' (', 1).replace(')', '') if 'Distrito' in x else '')
perfiles['cirscuncripcion_actividad'] = perfiles['links_Zona'].apply(lambda x: split(x, '\n', 1).replace('Circunscripción:', '') if 'Circunscripción' in x else '')

perfiles['cargo_genero_actividad'] = perfiles['links_Actividad'].apply(lambda x: x.split(' ')[0].replace(':', ''))
perfiles['genero'] = perfiles['cargo_genero_actividad'].apply(lambda x: 'F' if x == 'Senadora' or x == 'Diputada' else 'M')
perfiles['cargo'] = perfiles['cargo_genero_actividad'].apply(lambda x: 'Diputado' if x == 'Diputado' or x == 'Diputada' else 'Senador')

perfiles['suplente_sil_id'] = perfiles['links_suplenteidhref'].apply(sacaId)

## Actividad legislativa vigente (actividad)

In [59]:
actividades = perfiles[['desde_mandato', 'hasta_mandato', 'cargo']]
actividades.columns = ['desde', 'hasta', 'cargo']
actividades['tipo'] = 'Legislativa'
actividades['nivel_de_gobierno'] = 'Nacional'
actividades['ambito_de_gobierno'] = 'Legislativo'
actividades['personaId'] = perfiles.index
actividades['contactoId'] = perfiles.index

## Datos personales de actividad legislativa vigente (personas)

In [60]:
personas = perfiles[['nombre', 'apellido', 'fecha_nacimiento', 'entidad_nacimiento', 'ciudad_nacimiento', 'genero', 'sil_id']]

## Contacto de actividad legislativa vigente (contacto)

In [61]:
contactos = perfiles[['links_Ubicacion_en_la_camara', 'links_Correo_Electronico', 'links_Telefono_en_camara']]
contactos.columns = ['direccion', 'correo_electronico', 'telefono_fijo']

## Partido de actividad legislativa vigente (organizaciones)

In [62]:
organizaciones = perfiles[['links_Partido']].groupby('links_Partido').count().reset_index()
organizaciones.columns = ['nombre_corto']
organizaciones = organizaciones[organizaciones['nombre_corto'] != 'Sin Partido'].reset_index()
organizaciones['tipo'] = 'Partido'
organizaciones = organizaciones.iloc[:, -3:]

actividades['organizacionId'] = perfiles['links_Partido'].apply(lambda x: organizaciones[organizaciones['nombre_corto'] == x].index.values[0] if x != 'Sin Partido' else None)
perfiles['organizacionId'] = perfiles['links_Partido'].apply(lambda x: organizaciones[organizaciones['nombre_corto'] == x].index.values[0] if x != 'Sin Partido' else None)

## Lista de actividad legislativa vigente (listas)

In [63]:
listas = perfiles[['links_Principio_de_eleccion', 'entidad_actividad', 'distrito_actividad', 'municipalidad_actividad', 'cirscuncripcion_actividad', 'organizacionId']].copy()
listas.columns = ['principio_eleccion', 'entidad', 'distrito', 'municipalidad', 'circunscripcion', 'organizacionId']

actividades['listaId'] = perfiles.index

# Suplentes
## Actividad  de suplente de legislador (actividad)

In [64]:
actividades_suplentes = perfiles[['desde_mandato', 'hasta_mandato']]
actividades_suplentes.columns = ['desde', 'hasta']
actividades_suplentes['tipo'] = 'Partidaria'
actividades_suplentes['nivel_de_gobierno'] = 'Nacional'
actividades_suplentes['ambito_de_gobierno'] = 'Legislativo'
actividades_suplentes['cargo'] = 'Suplente de ' + perfiles['cargo'].str.lower()
actividades_suplentes['personaId'] = len(perfiles) + perfiles.index
actividades_suplentes['contactoId'] = len(perfiles) + perfiles.index
actividades_suplentes['organizacionId'] = perfiles['links_Partido'].apply(lambda x: organizaciones[organizaciones['nombre_corto'] == x].index.values[0] if x != 'Sin Partido' else None)
actividades_suplentes['listaId'] = perfiles.index

actividades['suplenteId'] = len(perfiles) + perfiles.index

actividades = pd.concat([actividades, actividades_suplentes]).reset_index().iloc[:, 1:]

## Datos personales de suplente de legislador (personas)

In [65]:
personas_suplentes = perfiles[['suplente_nombre', 'suplente_apellido', 'genero', 'suplente_sil_id']]
personas_suplentes.columns = ['nombre', 'apellido', 'genero', 'sil_id']

personas = pd.concat([personas, personas_suplentes]).reset_index().iloc[:, 1:]

# Actividad en comisiones
## Preprocesamiento

In [66]:
fila_comisiones = pd.read_csv('data/fila_comisiones.csv')

fila_comisiones['sil_id'] = fila_comisiones['url'].apply(lambda x: x.split('&Referencia=')[1]).astype(int)
fila_comisiones = fila_comisiones[fila_comisiones['sil_id'].isin(perfiles['sil_id'])]
fila_comisiones = fila_comisiones[fila_comisiones['estatus'] == 'Activo']

def sacaDependencia(string):
    if pd.isnull(string):
        return ''
    result = re.search('\((.*)\)', string)
    if not result:
        return None
    return result.group(1).replace('C. ', '')

fila_comisiones['dependencia'] = fila_comisiones['comision'].apply(sacaDependencia)
fila_comisiones['comision_nombre'] = fila_comisiones['comision'].apply(lambda x: x.split(' (')[0])

## Comisiones (organizaciones)

In [67]:
org_comisiones = fila_comisiones[['comision', 'comision_nombre']].groupby('comision').first().reset_index()
org_comisiones.columns = ['nombre', 'nombre_corto']
org_comisiones['tipo'] = 'Comisión'

organizaciones = pd.concat([organizaciones, org_comisiones]).reset_index()
organizaciones = organizaciones.iloc[:, -3:]

## Pertenencia a comisiones (actividades)

In [68]:
actividad_comisiones = fila_comisiones[['fecha_inicial', 'fecha_inicial', 'puesto']]
actividad_comisiones.columns = ['desde', 'hasta', 'cargo']
actividad_comisiones['nivel_de_gobierno'] = 'Nacional'
actividad_comisiones['ambito_de_gobierno'] = 'Legislativo'
actividad_comisiones['tipo'] = 'Trabajo en Comisiones'
actividad_comisiones['personaId'] = fila_comisiones['sil_id'].apply(lambda x: personas[personas['sil_id'] == x].index.values[0])

actividad_comisiones['organizacionId'] = fila_comisiones['comision'].apply(lambda x: organizaciones[organizaciones['nombre'] == x].index.values[0])
actividades = pd.concat([actividades, actividad_comisiones])

# Actividad en órganos
## Preprocesamiento

In [69]:
fila_organos_de_gobierno = pd.read_csv('data/fila_organos_de_gobierno.csv')
fila_organos_de_gobierno['sil_id'] = fila_organos_de_gobierno['url'].apply(lambda x: x.split('&Referencia=')[1]).astype(int)
fila_organos_de_gobierno = fila_organos_de_gobierno[fila_organos_de_gobierno['sil_id'].isin(perfiles['sil_id'])]
fila_organos_de_gobierno = fila_organos_de_gobierno[fila_organos_de_gobierno['estatus'] == 'Activo']
fila_organos_de_gobierno['organo_nombre'] = fila_organos_de_gobierno['organo'].apply(lambda x: x.split(' L')[0].replace(',', '').strip())
fila_organos_de_gobierno['dependencia'] = fila_organos_de_gobierno['organo_nombre'].apply(lambda x: 'Cámara de Senadores' if 'Cámara de Senadores' in x else 'Cámara de Diputados' if 'Cámara de Diputados' in x else 'Comisión Permanente' if 'Comisión Permanente' in x else '')
fila_organos_de_gobierno['organo_nombre'] = fila_organos_de_gobierno['organo_nombre'].apply(lambda x: x.replace('Cámara de Senadores', '').replace('Cámara de Diputados', '').replace('Comisión Permanente', '').replace('de la', '').replace('-', '').strip())
fila_organos_de_gobierno['organo_nombre_largo'] = fila_organos_de_gobierno['organo_nombre'] + ' - ' + fila_organos_de_gobierno['dependencia']

## Pertenencia a órganos (actividades)

In [70]:
actividad_organos = fila_organos_de_gobierno[['fecha_inicial', 'fecha_inicial', 'puesto']]
actividad_organos.columns = ['desde', 'hasta', 'cargo']
actividad_organos['nivel_de_gobierno'] = 'Nacional'
actividad_organos['ambito_de_gobierno'] = 'Legislativo'
actividad_organos['tipo'] = 'Trabajo en Órganos'
actividad_organos['personaId'] = fila_organos_de_gobierno['sil_id'].apply(lambda x: personas[personas['sil_id'] == x].index.values[0])

## Órganos (organizaciones)

In [71]:
organos = fila_organos_de_gobierno[['organo_nombre', 'organo_nombre_largo']].groupby('organo_nombre_largo').first().reset_index()
organos.columns = ['nombre', 'nombre_corto']
organos['tipo'] = 'Órganos'
organizaciones = pd.concat([organizaciones, organos]).reset_index()
organizaciones = organizaciones.iloc[:, -3:]

actividad_organos['organizacionId'] = fila_organos_de_gobierno['organo_nombre_largo'].apply(lambda x: organizaciones[organizaciones['nombre'] == x].index.values[0])
actividades = pd.concat([actividades, actividad_organos])

# Guardar

In [72]:
personas.index.name = 'id'
personas.to_csv('data/output/personas.csv')

actividades.index.name = 'id'
actividades.to_csv('data/output/actividades.csv')

listas.index.name = 'id'
listas.to_csv('data/output/listas.csv')

organizaciones.index.name = 'id'
organizaciones.to_csv('data/output/organizaciones.csv')

contactos.index.name = 'id'
contactos.to_csv('data/output/contactos.csv')

In [73]:
personas = pd.read_csv('data/output/personas.csv')
actividades = pd.read_csv('data/output/actividades.csv')

actividades.drop(actividades[actividades['personaId'].isin(personas[personas['sil_id'].isnull()]['id'])].index, inplace=True)
personas.drop(personas[personas['sil_id'].isnull()].index, inplace=True)

personas.to_csv('data/output/personas.csv')
actividades.to_csv('data/output/actividades.csv')

# Estudios

In [74]:
academica = pd.read_csv('data/voluntarios/academica.csv')

academica['sil_id'] = academica['sil_id']
academica = academica.set_index('sil_id')

academica = academica[academica['ACT. SECT. PRIV'].isna()]
academica = academica[~academica[['Grado', 'Titulo', 'Institucion', 'SIGLAS']].isnull().all(axis=1)]

personas['sil_id'] = personas['sil_id'].astype(int)
personas = personas.set_index('sil_id')

estudios_org = academica.join(personas).reset_index()[['id', 'hasta', 'Grado', 'Titulo', 'Institucion', 'SIGLAS']]
estudios_org.columns = ['personaId', 'graduacion', 'grado', 'titulo', 'org-nombre', 'org-nombre-corto']
estudios_org['org-nombre'] = estudios_org['org-nombre'].str.strip()
estudios_org = estudios_org.fillna('')

## Universidades (organizaciones)

In [75]:
org_educacion = estudios_org.groupby(['org-nombre', 'org-nombre-corto']).count().iloc[:, :0].reset_index()
org_educacion.columns = ['nombre', 'nombre_corto']
org_educacion['tipo'] = 'Educación'

def matchOrg(x):
    match = org_educacion[org_educacion['nombre'] == x['org-nombre']]
    if len(match) > 0:
        return match.index.values[0]
    else:
        match = org_educacion[org_educacion['nombre_corto'] == x['org-nombre-corto']]
        if len(match) > 0:
            return match.index.values[0]
    return ''
    
estudios_org['organizacionId'] = estudios_org.apply(matchOrg, axis=1)
estudios_org['organizacionId'] = estudios_org['organizacionId'].apply(lambda x: x+len(organizaciones) if x != 0 else None)
estudios = estudios_org[['personaId', 'organizacionId', 'grado', 'titulo', 'graduacion']]

organizaciones = pd.concat([organizaciones, org_educacion]).iloc[:, 1:]

# Guardar

In [76]:
organizaciones.index.name = 'id'
organizaciones.to_csv('data/output/organizaciones.csv')

estudios.index.name = 'id'
estudios.to_csv('data/output/estudios.csv')

# Actividades previas

In [77]:
import pandas as pd
import numpy as np

administrativa = pd.read_csv('data/voluntarios/administrativa.csv')
administrativa['tabla'] = 'administrativa'

candidaturas = pd.read_csv('data/voluntarios/candidaturas.csv')
candidaturas['tabla'] = 'candidaturas'

docencia_investigacion = pd.read_csv('data/voluntarios/docencia_investigacion.csv')
docencia_investigacion['tabla'] = 'docencia_investigacion'

empresarial = pd.read_csv('data/voluntarios/empresarial.csv')
empresarial['tabla'] = 'empresarial'

legislativa = pd.read_csv('data/voluntarios/legislativa.csv')
legislativa['tabla'] = 'legislativa'

otros = pd.read_csv('data/voluntarios/otros.csv')
otros['tabla'] = 'otros'

partidaria = pd.read_csv('data/voluntarios/partidaria.csv')
partidaria['tabla'] = 'partidaria'

personas_base = pd.read_csv('data/output/personas.csv')
actividades_base = pd.read_csv('data/output/actividades.csv')
organizaciones_base = pd.read_csv('data/output/organizaciones.csv')
contactos_base = pd.read_csv('data/output/contactos.csv')

In [78]:
actividades_raw = pd.concat([administrativa, candidaturas, docencia_investigacion, empresarial, legislativa, otros, partidaria])
actividades_raw = actividades_raw[actividades_raw['IGNORAR'].isnull()]
actividades_raw = actividades_raw.reset_index().iloc[:, 1:].reset_index()

In [79]:
actividades = actividades_raw[['Cargo', 'Nivel de gobierno', 'Ámbito de gobierno', 'desde', 'hasta', 'tipo', 'tabla', 'pública', 'PUBLICA', 'PRIVADA', 'sil_id', 'index', 'actividad']]
organizaciones = actividades_raw[['Organización', 'SIGLAS', 'PARTIDO', 'Área']]
contactos = actividades_raw[['Ciudad donde ejerció el cargo', 'Estado donde ejerció el cargo']]

In [80]:
def tipoActividad(x):
    if x == 'candidaturas' or x == 'partidaria':
        return 'Partidaria'
    
    if x == 'docencia_investigacion' or x == 'empresarial' or x == 'otros' or x == 'administrativa':
        return 'Privada'
    
    if x == 'legislativa':
        return 'Pública'

actividades['tipo'] = actividades['tabla'].apply(tipoActividad)
actividades['tipo'] = actividades.apply(lambda x: 'Pública' if x['pública'] == 'X' or x['pública'] == 'x' else x['tipo'], axis=1)
actividades['tipo'] = actividades.apply(lambda x: 'Pública' if x['PUBLICA'] == 'X' or x['PUBLICA'] == 'x' else x['tipo'], axis=1)
actividades['tipo'] = actividades.apply(lambda x: 'Privada' if x['PRIVADA'] == 'X' or x['PRIVADA'] == 'x' else x['tipo'], axis=1)

personas_base['sil_id'] = personas_base['sil_id'].astype(int)
personas_base = personas_base.set_index('sil_id')

actividades = actividades[~actividades['Cargo'].isnull()]

actividades = actividades.set_index('sil_id').join(personas_base).reset_index()[['Cargo', 'tipo', 'Nivel de gobierno', 'Ámbito de gobierno', 'desde', 'hasta', 'index', 'id', 'actividad']]
actividades.columns = ['cargo', 'tipo', 'nivel_de_gobierno', 'ambito_de_gobierno', 'desde', 'hasta', 'index', 'personaId', 'actividad']

actividades = actividades.dropna(subset=['personaId'])

In [81]:
contactos = contactos.dropna(subset=['Ciudad donde ejerció el cargo', 'Estado donde ejerció el cargo'], how='all').reset_index()
contactos.columns = ['index', 'ciudad', 'estado']

In [82]:
def getIndex(x):
    indices = contactos[contactos['index'] == x.name].index.values
    if len(indices) >= 1:
        return indices[0]
    else:
        return np.nan

organizaciones['contactoId'] = organizaciones.apply(getIndex, axis=1)
organizaciones['contactoId'] = organizaciones['contactoId'].apply(lambda x: np.nan if pd.isnull(x) else x + len(contactos_base))

contactos_f = pd.concat([contactos_base, contactos]).reset_index()

In [83]:
organizaciones['Organización'] = organizaciones.apply(lambda x: x['Organización'] if pd.isnull(x['PARTIDO']) else x['PARTIDO'], axis=1)
organizaciones = organizaciones[['Organización', 'SIGLAS', 'contactoId']]
organizaciones.columns = ['nombre', 'nombre_corto', 'contactoId']
organizaciones = organizaciones.dropna(how='all').reset_index()

In [84]:
def addInd(group):
    group[1][1]['org_id'] = group[0]
    return group[1][1]
organizaciones = organizaciones.fillna('')
organizaciones = pd.concat([addInd(x) for x in enumerate(organizaciones.groupby(['nombre', 'nombre_corto']))])

In [85]:
def getIndex(x):
    org_id = organizaciones[organizaciones['index'] == int(x['index'])]['org_id'].values
    if len(org_id) >= 1:
        return org_id[0]
    else:
        return np.nan

actividades['organizacionId'] = actividades.apply(getIndex, axis=1)

organizaciones = organizaciones.groupby(['nombre', 'nombre_corto']).first().reset_index()

In [86]:
actividades['organizacionId'] = actividades['organizacionId'].apply(lambda x: np.nan if pd.isnull(x) else x + len(organizaciones_base))

organizaciones_f = pd.concat([organizaciones_base, organizaciones]).reset_index()
actividades_f = pd.concat([actividades_base, actividades]).reset_index()

In [ ]:
ejecutivo = ['EJEcutivo', 'Ejecutivo', 'Ejecutivo ', 'Ejecutivo/Autonomo', 'ejecutivo', 'ejecutivo ', 'ejecuttivo']
legislativo = ['Legislativo', 'Legisltivo', 'legislativo', 'legisltivo']
judicial = ['Judicial', 'Judicial ', 'Juridico', 'judicial', 'jurídico']
autonomo = ['Auronomo ', 'Autonomo', 'Organismo Autónomo', 'Organismo autónomo', 'Organismo autónomo ', 'organismo autónomo', 'organismos autonomos', 'organismos autónomos', 'organización autonoma ']

actividades['ambito_de_gobierno'] = actividades['ambito_de_gobierno'].apply(lambda x: 1 if x in ejecutivo else 2 if x in legislativo else 3 if x in judicial else 4 if x in autonomo else x)

nacional = ['Nacional', 'nacional', 'nacional ']
estatal = ['Estadal', 'Estatal', 'Estatal ', 'estatal', 'estudual', 'estudual ']
municipal = ['Municipal', 'Munipal', 'mucipal', 'mucipal ', 'municiapl', 'municipal', 'municipal ']

actividades['nivel_de_gobierno'] = actividades['nivel_de_gobierno'].apply(lambda x: 1 if x in nacional else 2 if x in estatal else 3 if x in municipal else x)

# Guardar

In [87]:
contactos_f.index.name = 'id'
contactos_f.to_csv('data/output/contactos.csv')

organizaciones_f.index.name = 'id'
organizaciones_f.to_csv('data/output/organizaciones.csv')

actividades_f.index.name = 'id'
actividades_f.to_csv('data/output/actividades.csv')

# Abrir, limpiar y guardar

In [88]:
personas = pd.read_csv('data/output/personas.csv')
personas[['id', 'apellido', 'ciudad_nacimiento', 'entidad_nacimiento', 'fecha_nacimiento', 'genero', 'nombre', 'sil_id']].to_csv('data/output/personas.csv', index=False)

actividades = pd.read_csv('data/output/actividades.csv')
actividades[['id', 'ambito_de_gobierno', 'cargo', 'contactoId', 'desde', 'hasta', 'listaId', 'nivel_de_gobierno', 'organizacionId', 'personaId', 'suplenteId', 'tipo']].to_csv('data/output/actividades.csv', index=False)

listas = pd.read_csv('data/output/listas.csv')
listas[['id', 'principio_eleccion', 'entidad', 'distrito', 'municipalidad', 'circunscripcion', 'organizacionId']].to_csv('data/output/listas.csv', index=False)

organizaciones = pd.read_csv('data/output/organizaciones.csv')
organizaciones[['id', 'contactoId', 'nombre', 'nombre_corto', 'tipo']].to_csv('data/output/organizaciones.csv', index=False)

contactos = pd.read_csv('data/output/contactos.csv')
contactos[['id', 'ciudad', 'correo_electronico', 'direccion', 'estado', 'telefono_fijo']].to_csv('data/output/contactos.csv', index=False)

estudios = pd.read_csv('data/output/estudios.csv')
estudios[['id', 'personaId', 'organizacionId', 'grado', 'titulo', 'graduacion']].to_csv('data/output/estudios.csv', index=False)